---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [35]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle


---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [36]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [37]:
def degree_distribution(G):
    degrees = G.degree()
    degree_values = sorted(set(degrees.values()))
    histogram = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G))
                for i in degree_values]
    return histogram

def graph_identification():
    results = []
    for G in P1_Graphs:
        cluster = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        degree_dist = degree_distribution(G)
        
        if len(degree_dist)>10:
            results.append('PA')
        elif cluster < 0.1:
            results.append('SW_H')
        else:
            results.append('SW_L')
    
    return results

graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [38]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [45]:
def salary_predictions():
    from sklearn.model_selection import train_test_split 
    
    #     Question 3 assignment 3 applied text
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.metrics import roc_auc_score
    from sklearn.feature_extraction.text import CountVectorizer

    #     Question 3 assignment 3 applied text
    
    df = pd.DataFrame(index=G.nodes())
    
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
#     https://datatofish.com/rows-with-nan-pandas-dataframe/
   
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
#     df['closeness'] = pd.Series(nx.closeness_centrality(G))
    
#     df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized=True, endpoints=False))

    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['centrality'] = pd.Series(nx.degree_centrality(G))
    
    df_nan = df[df['ManagementSalary'].isnull()]
    
    indexes_to_drop = df_nan.index.tolist()
    
    df_data = df.drop(indexes_to_drop)
  
    
#     NEED TO SPLIT THE DATAFRAME INTO 2
#  ONE WHERE THERE IS A MANAGEMENT SALARY (WHICH IS WHAT TO TRAIN ON )
#  AND THE OTHER WHERE THERE IS NOT A AMANAGEMENT SALARY
#  THEN BASED ON THE FEATURES, CALCULATE THE PROBABLITY THAT THE INDIVIDUAL WILL HAVE A MANAGEMENT SALARY
    
    X = df.iloc[:,1:]
    y = df.iloc[:,0]

    X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=0)
    
#     CHECK OUT ASSIGNMETN 4 FOR APPLIED MACHINE LEARNING FOR HOW TO HANDLE PREDICTIONS? 
    
#     model = MultinomialNB(alpha=0.1)
    
#     vect = CountVectorizer().fit(X_train)
    
#     X_train_vectorized = vect.transform(X_train)
    
#     X_test_vectorized = vect.transform(X_test)
    
#     model.fit(X_train_vectorized, y_train)

#     predictions = model.predict(X_test_vectorized)
    
#     answer = roc_auc_score(y_test, predictions)
    
    return df_data
salary_predictions()

ValueError: Found input variables with inconsistent numbers of samples: [252, 753]

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here